# This is a sandbox for setting up a run of the simulation on `TNC` only. 

* It can be run on a small subsample pandas DataFrame


In [14]:
import pandas as pd
import numpy as np
import replicaEVSE.load_curve as sim
import os
import joblib
import dask.dataframe as dd

%reload_ext autoreload
%autoreload 2



pd.set_option('display.max_columns', None)

datadir = '../../data'
mode = 'ON_DEMAND_AUTO'
simulation_id = 'dev'
test = True

#Created in the EIA_data_download.ipynb notebook
existing_load=pd.read_csv('../../data/EIA_demand_summary.csv') 
if test:
    # df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_subsample.parquet'))
    df = dd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted.parquet'))
    # df = df.loc[df['mode'] == mode]
    df = df.sample(frac=0.001, random_state=1)
    # df = df.sort_values(by=['person_id', 'start_time']).reset_index(drop=True)
    simulation_id = 'dev'

else: 


    # read in the joined trips and population data sets
    merged_df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted.parquet'))

    # right now, only look at private auto trips
    df = merged_df.loc[merged_df['mode'] == mode]



In [17]:
df['mode'].unique().compute()

0              WALKING
1              CARPOOL
2         PRIVATE_AUTO
3           COMMERCIAL
4       ON_DEMAND_AUTO
5    OTHER_TRAVEL_MODE
6       PUBLIC_TRANSIT
7               BIKING
Name: mode, dtype: object

In [20]:
df = df.loc[df['mode'] == 'ON_DEMAND_AUTO']
df = df.sort_values(by=['person_id', 'start_time']).reset_index(drop=True)
df = df.compute()

### figure out how to determine P(charge) and how many trips that will take. 

In [28]:
# this is the instance where we see how many trips it took to get to 
tot_distance = 0
for row in len(df)
    trip = df.iloc[row,]
    tot_distance += trip['distance_miles']
    if tot_distance > 100:
        tot_distance = 0

    


,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type
0,10387203185911262793,100015405478630305,ON_DEMAND_AUTO,HOME,SHOP,0 days 17:54:33,0 days 18:16:57,2.593752,nan,530330075001,47.622594,-122.314176,530330063001,47.641060,-122.270055,commercial,retail,commercial,retail,residential,single_family,residential,single_family,47.623460,-122.312882,47.640022,-122.277946,saturday,17065429525948427972,530330063001,nan,nan,53033006300,nan,nan,65_plus,72.0,F,white,not_hispanic_or_latino,100000_plus,258486.0,notinlf,bachelors_degree,not_attending_school,not_working,head_of_household,0,nan,not_working,owner,same_house,1_person,125000_plus,258486.0,living_alone,1,single_family,core,english,47.640022,-122.277946,NaN,NaN,NaN,NaN,under_16_or_notinlf,HOME
1,10340519869883566639,10003942663675567783,ON_DEMAND_AUTO,RECREATION,HOME,0 days 16:49:14,0 days 17:23:11,16.544300,nan,410510072021,45.614080,-122.729634,530110413212,45.628324,-122.485804,residential,multi_family,residential,multi_family,mixed_use,open_space,commercial,non_retail_attraction,45.599209,-122.653974,45.631740,-122.478512,thursday,8674205008720373855,410510072021,530110411083,nan,41051007202,53011041108,nan,65_plus,73.0,M,white,not_hispanic_or_latino,40000_80000,61167.0,employed,bachelors_degree,not_attending_school,naics48_49,relative,0,nan,driving,renter,same_house,2_person,125000_plus,128661.0,family_single,2,multiple_units,core,english,45.599209,-122.653974,45.652338,-122.596624,NaN,NaN,worked_in_person,PUBLIC
2,17532146266495730744,10014319498368367973,ON_DEMAND_AUTO,HOME,SOCIAL,0 days 22:15:06,0 days 22:25:34,6.153143,nan,530330026003,47.677457,-122.311997,530330093002,47.587222,-122.335316,mixed_use,multi_family,residential,multi_family,mixed_use,multi_family,residential,multi_family,47.676171,-122.316434,47.595987,-122.333574,thursday,6748284731740869040,530330093002,530330093002,nan,53033009300,53033009300,nan,35_64,37.0,M,white,not_hispanic_or_latino,100000_plus,133589.0,employed,bachelors_degree,not_attending_school,naics54,head_of_household,0,nan,walking,renter,within_us,1_person,125000_plus,133589.0,living_alone,zero,multiple_units,core,english,47.595987,-122.333574,47.590614,-122.333689,NaN,NaN,worked_from_home,HOME
3,7182792280722927872,1002332960170982594,ON_DEMAND_AUTO,MAINTENANCE,HOME,0 days 11:00:32,0 days 11:29:55,18.595010,nan,530610519184,47.783952,-122.213716,530330053011,47.663063,-122.308675,residential,single_family,residential,single_family,civic_institutional,civic_institutional,civic_institutional,civic_institutional,47.788908,-122.215361,47.662188,-122.307517,saturday,14644697824760795458,530610519184,530330058014,nan,53061051918,53033005801,nan,18_24,18.0,M,two_or_more_races,hispanic_or_latino,20000_40000,36100.0,employed,k_12,not_attending_school,naics238220,child,0,nan,driving,renter,same_house,4_person,125000_plus,181535.0,family_single,2,single_family,core,english,47.788908,-122.215361,47.661479,-122.385466,NaN,NaN,worked_from_home,PUBLIC
4,857207562832207380,10027802285380474499,ON_DEMAND_AUTO,HOME,OTHER_ACTIVITY_TYPE,0 days 21:23:36,0 days 21:42:39,15.275144,nan

run the simulation on small dataset to test. 

In [ ]:
# this takes ~1min for 10,000 trips
# out = sim.simulate_person_load(df, existing_load, 'base', managed=False)

embarrassingly parallel

In [13]:
number_of_chunks = 10
df_list = np.array_split(df, number_of_chunks)

In [24]:
# run the simulation in parallel
# df must be a pandas dataframe
charge_sims = joblib.Parallel(verbose=10, n_jobs=-1)(
    joblib.delayed(sim.simulate_person_load)(
    trips_df=df,
    existing_load=existing_load,
    simulation_id=simulation_id,
    managed=False
) for df in df_list)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    5.2s remaining:   12.1s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    5.3s remaining:    5.3s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    5.3s remaining:    2.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    5.5s finished


### Restack the DataFrames

In [18]:
# restack the dataframes
charges_list = [x['charges'] for x in charge_sims]
loads_list = [x['loads'] for x in charge_sims]

In [21]:
charges_list

AttributeError: 'list' object has no attribute 'compute'

Pandas significantly faster since we can fit these in memory. 

In [16]:
# restack the dataframes
charges_df = pd.concat(charges_list)
loads_df = pd.concat(loads_list) # huge ~200 million rows

TypeError: cannot concatenate object of type '<class 'dask.delayed.Delayed'>'; only Series and DataFrame objs are valid

In [ ]:
loads_df

,person_id,load_segment_id,charge_id,window_start_time,window_end_time,load_kW
0,1000016319449981116,7318818662359676746_base_0,7318818662359676746_base,0 days 00:00:00,0 days 01:00:00,7.200000
1,1000016319449981116,7318818662359676746_base_1,7318818662359676746_base,0 days 01:00:00,0 days 02:00:00,7.200000
2,1000016319449981116,7318818662359676746_base_2,7318818662359676746_base,0 days 02:00:00,0 days 03:00:00,1.647283
3,1000016319449981116,7318818662359676746_base_3,7318818662359676746_base,0 days 03:00:00,0 days 04:00:00,0.000000
4,1000016319449981116,7318818662359676746_base_4,7318818662359676746_base,0 days 04:00:00,0 days 05:00:00,0.000000
...,...,...,...,...,...,...
19,9999319521751741792,1701538494003822315_base_19,1701538494003822315_base,0 days 19:00:00,0 days 20:00:00,0.000000
20,9999319521751741792,1701538494003822315_base_20,1701538494003822315_base,0 days 20:00:00,0 days 21:00:00,0.000000
21,9999319521751741792,1701538494003822315_base_21,1701538494003822315_base,0 days 21:00:00,0 days 22:00:00,0.000000
22,9999319521751741792,1701538494003822315_base_22,1701538494003822315_base,0 days 22:00:00,0 days 23:00:00,0.000000


In [ ]:
charges_df.to_parquet(os.path.join(datadir, f'charges_{mode}_{simulation_id}.parquet'), overwrite=True)
loads_df.to_parquet(os.path.join(datadir, f'loads_{mode}_{simulation_id}.parquet'), overwrite=True)

NameError: name 'charges_df' is not defined